In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Load the dataset
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
# Merge the ratings and movies datasets
data = pd.merge(ratings, movies, on='movieId')

In [ ]:
# Create a matrix of user-item interactions
user_item_matrix = data.pivot_table(index='userId', columns='title', values='rating')

# Replace missing values with 0
user_item_matrix.fillna(0, inplace=True)

In [ ]:
# Calculate the cosine similarity between users
similarity_matrix = cosine_similarity(user_item_matrix)

In [ ]:
# Function to get recommendations for a user
def get_recommendations(user_id, num_recommendations):
    # Get the index of the user
    user_index = user_item_matrix.index.get_loc(user_id)

    # Get the similarity scores for the user
    similarity_scores = similarity_matrix[user_index]

    # Get the indices of the top similar users
    top_similar_users = similarity_scores.argsort()[-num_recommendations-1:-1][::-1]

    # Get the movie recommendations
    recommendations = []
    for similar_user_index in top_similar_users:
        similar_user_id = user_item_matrix.index[similar_user_index]
        similar_user_ratings = user_item_matrix.loc[similar_user_id]
        top_movies = similar_user_ratings.argsort()[-num_recommendations:][::-1]
        for movie_index in top_movies:
            movie_title = user_item_matrix.columns[movie_index]
            if movie_title not in recommendations and user_item_matrix.loc[user_id, movie_title] == 0:
                recommendations.append(movie_title)

    return recommendations[:num_recommendations]

In [ ]:
# Test the recommendation function
user_id = 1
num_recommendations = 5
recommendations = get_recommendations(user_id, num_recommendations)
print("Recommendations for user", user_id, ":", recommendations)